# 0. Install Dependencies

In [4]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     |████████████████████████████████| 320.4 MB 41 kB/s 
     |████████████████████████████████| 459 kB 39.7 MB/s 
     |████████████████████████████████| 20.1 MB 17.8 MB/s 
     |████████████████████████████████| 2.9 MB 11.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:


     |████████████████████████████████| 52 kB 654 kB/s 


# 1. Test Random Environment with OpenAI Gym

In [45]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import sympy

In [46]:
a = np.array([0,1,2,3,4,5,6,7,8,9])
a[(6 - 6%4):(6 - 6%4 + 4)] = [0,0,0,0]
a

array([0, 1, 2, 3, 0, 0, 0, 0, 8, 9])

In [47]:
sample_state = [
    [1, 2, -3], # 1x + 2y - 3 = 0   <- first equation
    [3, 0, -6]  # 3x - 6 = 0        <- second equation
]
def state_to_equation(eqNr):
    state = sample_state[eqNr]
    left = "{} * x + {} * y + {}".format(*state)
    return left, 0
    
left1, right1 = state_to_equation(0)
eq1 = sympy.Eq(sympy.sympify(left1), right1)
print(eq1)
               
left2, right2 = state_to_equation(1)
eq2 = sympy.Eq(sympy.sympify(left2), right2)

x,y = sympy.symbols('x y')
result = sympy.linsolve([eq1, eq2], (x, y))
result

Eq(x + 2*y - 3, 0)


FiniteSet((2, 1/2))

In [48]:
import math
import random
from functools import reduce
from sympy.ntheory import factorint
from scipy.stats import norm
lista = [15, 30, 10]

In [49]:
from time import perf_counter as pc

In [50]:
Discrete?

In [60]:
class MathEquationEnv(Env):
    
    def __init__(self):
        self.action_space = Discrete(12)
        self.observation_space = Discrete(6)
        self.state = self.random_state()
        
        ###R
        self.previous_state_value = 0
        self.pretty_denominators = [1,2,3,4,5,10,20,25,100]
        self.previous_step_denominators = [0,0]
        self.previous_step_solvable = 0
        ###

        # print("Initial state:")
        # print(self.state_to_equation(0)[0])
        # print(self.state_to_equation(1)[0])
        # print()

        
        self.generator_length = 60
        x,y = sympy.symbols('x y')
        self.x = x
        self.y = y
    
    def random_state(self):
        random_state = random.sample(range(-10, 10), 6)

#         random_state[0] = 0
#         random_state[2] = 0
        
#         random_state[5] = 0
#         random_state[7] = 0
        
# #         random_state[4] = 0
# #         random_state[9] = 0
#         #print(random_state)
        return random_state
    
    def state_to_equation(self, eqNr):
        eq_state = self.state[3*eqNr:3*(eqNr+1)]
        left = "{} * x + {} * y + {} * 1".format(*eq_state)
        #print(left)
        return left, 0
            
    def step(self, action):
        """A step
        """

        
        self.generator_length-=1
        info = {}
        reward = 0
        # if action in (0,2,5,7,10,12,15,17):
        #   action = random.choice((1,3,4,6,8,9))
        self.state[action%6] += 1 if action//10 else -1
            
            
        #x,y = sympy.symbols('x y')
        left1, right1 = self.state_to_equation(0)
        eq1 = sympy.Eq(sympy.sympify(left1), right1)
                
        left2, right2 = self.state_to_equation(1)
        eq2 = sympy.Eq(sympy.sympify(left2), right2)
        
        done = False      

        ##ending was here
        
        #print(sympy.sympify(left1), right1)
        #print(sympy.sympify(left2), right2)
        #print('RESULT', eq1, eq2)
        if eq1 == True or eq2 == True or eq1 == False or eq2 == False:
            return self.state, reward, done, info
        t0 = pc()
        try:
            t0 = pc()
            # result = sympy.nsolve([eq1, eq2], (self.x, self.y), (1,1))
            result = sympy.solve([eq1, eq2], (self.x, self.y))
            # print(result, type(result))
            # print(pc() - t0,"\n", eq1, "\n", eq2, type(eq1))
        except Exception:
            result = None

        ###R
        n_of_equations = 2

        coeffs = [self.state[3*x:3*(x+1)] for x in range(n_of_equations)]
        # coeffs1 = self.state[:5]
        # coeffs2 = self.state[5:]
        
        gcds = [reduce(lambda a,b: math.gcd(a,b), coeffs_reduce) for coeffs_reduce in coeffs]
        # gcd_equation1 = reduce(lambda a,b,: math.gcd(a,b), coeffs1)
        # gcd_equation2 = reduce(lambda a,b,: math.gcd(a,b), coeffs2)
        
        number_of_divisors = [sum(factorint(gcd).values()) for gcd in gcds]
        # numbers_of_divisors1 = sum(factorint(gcd_equation1).values())
        # numbers_of_divisors2 = sum(factorint(gcd_equation2).values())
        
        value1 = sum( [ norm(6, 2).cdf(divisor_number) for divisor_number in number_of_divisors] ) * 10
        
        reward += ( value1 - self.previous_state_value)

        self.previous_state_value = value1

        try:
          if result:
            if result[self.x].q in self.pretty_denominators:
              if self.previous_step_denominators[0] != 1:
                reward += 1
                self.previous_step_denominators[0] = 1
            else:
              if self.previous_step_denominators[0] != -1:
                reward -= 1
                self.previous_step_denominators[0] = -1

            if result[self.y].q in self.pretty_denominators:
              if self.previous_step_denominators[1] != 1: 
                reward += 1
                self.previous_step_denominators[1] = 1
            else:
              if self.previous_step_denominators[1] != -1:
                reward -= 1
                self.previous_step_denominators[1] = -1
        except:
          print("Yikes", result)
        
         

        
        # print(value, value - self.previous_state_value)
        
        # value =+ norm(6, 2).cdf(numbers_of_divisors1)
        # value =+ norm(6, 2).cdf(numbers_of_divisors2)
        
        ## - Harder than I thought (Will have to do some validation beforehand)
        # if isinstance(tuple, result):
        #   pretty_results = sum( map(lambda x: 1 if x.q in self.pretty_denominators else 0, result) )
        # elif isinstance(dict, result):
        #   ##implement(easier than the first one)
        #   pass

        ###

        if self.generator_length <= 0:
            print("Final state:")
            print(self.state_to_equation(0)[0])
            print(self.state_to_equation(1)[0])
            print(result)
            done = True

        #print(result)
        if not result:
            if self.previous_step_solvable != -1:
              reward -= 1
              self.previous_step_solvable = -1
        else:
            if self.previous_step_solvable != 1:
              reward += 1
              self.previous_step_solvable = 1
            # print("Final state:")
            # print(self.state_to_equation(0)[0])
            # print(self.state_to_equation(1)[0])
            # print()
            # done = True
        
        ###R
        # self.previous_step_state[ action % 10 ] += 1 if action // 10 else -1
        ###
            
        return self.state, reward, done, info
        
    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.state = self.random_state()
        print("Initial state:")
        print(self.state_to_equation(0)[0])
        print(self.state_to_equation(1)[0])
        print()
        self.generator_length = 60
        return self.state

In [61]:
env = MathEquationEnv()

In [62]:
env.observation_space.sample()

3

In [63]:
x,y = 0.522022196614492, -0.915915828629164
print(8 * x**2 + 5*x -6*y**2 -9*y -8)

1.7763568394002505e-15


In [64]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        #print('hmm')
        action = env.action_space.sample()
        
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Initial state:
-5 * x + -10 * y + -3 * 1
-6 * x + 0 * y + 6 * 1

Final state:
-14 * x + -21 * y + -15 * 1
-18 * x + -2 * y + 8 * 1
{x: 99/175, y: -191/175}
Episode:1 Score:1.075595633574062
Initial state:
3 * x + -2 * y + -3 * 1
8 * x + 2 * y + 9 * 1

Final state:
-7 * x + -14 * y + -10 * 1
0 * x + 8 * y + 14 * 1
{x: 29/14, y: -7/4}
Episode:2 Score:1.0
Initial state:
9 * x + -1 * y + -9 * 1
-5 * x + 0 * y + 6 * 1

Final state:
-1 * x + -11 * y + -17 * 1
-16 * x + 0 * y + 1 * 1
{x: 1/16, y: -273/176}
Episode:3 Score:-1.0485976729414603
Initial state:
-1 * x + 3 * y + -4 * 1
8 * x + 4 * y + 7 * 1

Final state:
-13 * x + -5 * y + -14 * 1
-5 * x + 5 * y + 7 * 1
{x: -7/18, y: -161/90}
Episode:4 Score:0.0
Initial state:
1 * x + 0 * y + 2 * 1
-4 * x + -5 * y + 4 * 1

Final state:
-5 * x + -11 * y + -9 * 1
-12 * x + -9 * y + 8 * 1
{x: 169/87, y: -148/87}
Episode:5 Score:0.0
Initial state:
-10 * x + 8 * y + 2 * 1
-1 * x + 4 * y + -5 * 1

Final state:
-18 * x + -5 * y + -4 * 1
-14 * x + 1 * y + 

# 2. Create a Deep Learning Model with Keras

In [65]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [66]:
env.observation_space

Discrete(6)

In [67]:
states = env.observation_space.shape
print(states)
actions = env.action_space.n

()


In [68]:
env.action_space

Discrete(12)

In [69]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(6, activation='relu', input_shape=(1,6) )  )
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.add(Flatten())
    return model

In [70]:
del model 

In [71]:
model = build_model(states, actions)

In [72]:
model.output_shape

(None, 12)

In [73]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1, 6)              42        
_________________________________________________________________
dense_10 (Dense)             (None, 1, 24)             168       
_________________________________________________________________
dense_11 (Dense)             (None, 1, 12)             300       
_________________________________________________________________
flatten_3 (Flatten)          (None, 12)                0         
Total params: 510
Trainable params: 510
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [74]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [75]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [76]:
tuple(model.input.shape)

(None, 1, 6)

In [77]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)

Training for 5000 steps ...
Initial state:
-9 * x + -5 * y + 9 * 1
-3 * x + 3 * y + 1 * 1

Interval 1 (0 steps performed)
   11/10000 [..............................] - ETA: 10:04 - reward: 0.0000e+00

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   59/10000 [..............................] - ETA: 12:26 - reward: -8.2369e-04Final state:
-45 * x + -11 * y + 9 * 1
-4 * x + 4 * y + 17 * 1
{x: 223/224, y: -729/224}
   60/10000 [..............................] - ETA: 12:24 - reward: -8.0996e-04Initial state:
4 * x + 2 * y + -8 * 1
7 * x + 1 * y + -5 * 1

  119/10000 [..............................] - ETA: 10:53 - reward: -4.0838e-04Final state:
-22 * x + -28 * y + -9 * 1
7 * x + 1 * y + -4 * 1
{x: 121/174, y: -151/174}
  120/10000 [..............................] - ETA: 10:53 - reward: -4.0498e-04Initial state:
-10 * x + -8 * y + -2 * 1
1 * x + 8 * y + 9 * 1

  179/10000 [..............................] - ETA: 10:13 - reward: -2.7150e-04Final state:
-40 * x + -30 * y + -2 * 1
1 * x + 8 * y + 17 * 1
{x: 247/145, y: -339/145}
  180/10000 [..............................] - ETA: 10:14 - reward: 3.0840e-19 Initial state:
7 * x + -9 * y + 2 * 1
4 * x + 6 * y + -7 * 1

  238/10000 [..............................] - ETA: 9:25 - reward: -2.0

In [78]:
scores = dqn.test(env, nb_episodes=50, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 50 episodes ...
Initial state:
-8 * x + 7 * y + -9 * 1
1 * x + -10 * y + 6 * 1

Final state:
-8 * x + 7 * y + -69 * 1
1 * x + -10 * y + 6 * 1
{x: -648/73, y: -21/73}
Episode 1: reward: 0.000, steps: 60
Initial state:
1 * x + 9 * y + -3 * 1
3 * x + -6 * y + 7 * 1

Final state:
1 * x + 9 * y + -63 * 1
3 * x + -6 * y + 7 * 1
{x: 105/11, y: 196/33}
Episode 2: reward: 0.000, steps: 60
Initial state:
8 * x + -4 * y + -1 * 1
0 * x + -7 * y + 1 * 1

Final state:
8 * x + -4 * y + -61 * 1
0 * x + -7 * y + 1 * 1
{x: 431/56, y: 1/7}
Episode 3: reward: 0.000, steps: 60
Initial state:
7 * x + -10 * y + 6 * 1
9 * x + -7 * y + -6 * 1

Final state:
-26 * x + -37 * y + 6 * 1
9 * x + -7 * y + -6 * 1
{x: 264/515, y: -102/515}
Episode 4: reward: 0.000, steps: 60
Initial state:
4 * x + -2 * y + 6 * 1
-3 * x + -1 * y + 2 * 1

Final state:
4 * x + -36 * y + 6 * 1
-3 * x + -1 * y + 28 * 1
{x: 501/56, y: 65/56}
Episode 5: reward: 0.049, steps: 60
Initial state:
-5 * x + 4 * y + -7 * 1
5 * x + -2 * y

# 4. Reloading Agent from Memory

In [ ]:
import gym

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

NameError: ignored

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

NameError: ignored

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
